In [2]:
import os
import json

In [4]:
with open(r"C:\Users\corii\OneDrive\Desktop\kaggle.json", 'r') as f:
    kaggle_credentials = json.load(f)

In [6]:
#Set the environment variables for authentication 
os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

#Check if Credentials are set
print("Kaggle credentials are set")

Kaggle credentials are set


In [18]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import time

# Load the CSV files using pandas
 
dec_path = "2019-Dec-Cleaned.csv"  

# Start timer
start_time = time.time()

#read in csv file
dec_df = pd.read_csv(dec_path)

# Print time taken to load data
print(f"Data loaded in {time.time() - start_time} seconds")

Data loaded in 2.587357759475708 seconds


In [20]:
# Load the CSV files using pandas
nov_path = "2019-Nov-Cleaned.csv"  

# Start timer
start_time = time.time()

nov_df = pd.read_csv(nov_path)

# Print time taken to load data
print(f"Data loaded in {time.time() - start_time} seconds")

Data loaded in 76.59385013580322 seconds


In [22]:
print(nov_df.head())
print(dec_df.head())

                event_time event_type  product_id          category_id  \
0  2019-11-01 00:00:00 UTC       view     1003461  2053013555631882655   
1  2019-11-01 00:00:00 UTC       view     5000088  2053013566100866035   
2  2019-11-01 00:00:01 UTC       view     3601530  2053013563810775923   
3  2019-11-01 00:00:01 UTC       view     1004775  2053013555631882655   
4  2019-11-01 00:00:01 UTC       view     1306894  2053013558920217191   

               category_code   brand   price    user_id  \
0     electronics.smartphone  xiaomi  489.07  520088904   
1  appliances.sewing_machine  janome  293.65  530496790   
2  appliances.kitchen.washer      lg  712.87  518085591   
3     electronics.smartphone  xiaomi  183.27  558856683   
4         computers.notebook      hp  360.09  520772685   

                           user_session  
0  4d3b30da-a5e4-49df-b1a8-ba5943f1dd33  
1  8e5f4f83-366c-4f70-860e-ca7417414283  
2  3bfb58cd-7892-48cc-8020-2f17e6de6e7f  
3  313628f1-68b8-460d-84f6-cec7a

In [24]:
# Get column names 
nov_columns = nov_df.columns.tolist()
print(f"November dataset columns: {nov_columns}")
dec_columns = dec_df.columns.tolist()
print(f"December dataset columns: {dec_columns}")

November dataset columns: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']
December dataset columns: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']


In [28]:
#Choose features
features = ['event_type', 'price']

In [30]:
# Merge both dataframes for clustering
combined_df = pd.concat([nov_df[features], dec_df[features]])

In [48]:
#Create mapping for categorical data in event type 
event_type_mapping = {'view': 0, 'cart': 1, 'purchase': 2}
combined_df['event_type'] = combined_df['event_type'].map(event_type_mapping)
print(combined_df[['event_type', 'event_type_encoded']].head(5))

   event_type  event_type_encoded
0           0                   0
1           0                   0
2           0                   0
3           0                   0
4           0                   0


In [50]:
combined_df

,event_type,price,event_type_encoded
0,0,489.07,0
1,0,293.65,0
2,0,712.87,0
3,0,183.27,0
4,0,360.09,0
...,...,...,...
2778100,0,591.75,0
2778101,0,275.54,0
2778102,0,5.92,0
2778103,0,1161.68,0


In [52]:
# Normalize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(combined_df)

In [56]:
#Run K-Means Cluster
kmeans = KMeans(n_clusters=3, random_state=42)  # Replace with desired number of clusters
kmeans.fit(scaled_data)

C:\Users\corii\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3, random_state=42)

In [76]:
#Assign Clusters to dataframe 
combined_df['cluster'] = kmeans.predict(scaled_data)

In [78]:
#View predictions 
print(combined_df[['price', 'event_type', 'cluster']].head())

    price  event_type  cluster
0  489.07           0        2
1  293.65           0        2
2  712.87           0        0
3  183.27           0        2
4  360.09           0        2


In [82]:
#Summary Statistics 
#Cluster Centroids
centroids = kmeans.cluster_centers_  # Centroids of the clusters
print("\nCluster Centroids:")
#updating based on number of features in centroids 
num_features = centroids.shape[1]
column_names = [f'Feature {i+1}' for i in range(num_features)]
centroids_df = pd.DataFrame(centroids, columns=column_names)
print(centroids_df)

# Cluster sizes
cluster_sizes = combined_df['cluster'].value_counts()
print("\nCluster Sizes:")
print(cluster_sizes)

# Statistical summary for each cluster
print("\nStatistical Summary for Each Cluster:")
cluster_stats = combined_df.groupby('cluster')[['event_type', 'price']].describe()
print(cluster_stats)

#Evaluate Inertia
print(f"Within Set Sum of Squared Errors (WSSSE) = {kmeans.inertia_}")


Cluster Centroids:
   Feature 1  Feature 2  Feature 3
0  -0.250239   1.799328  -0.250239
1   3.546227   0.002631   3.546227
2  -0.250239  -0.387727  -0.250239

Cluster Sizes:
cluster
2    34483992
0     7426282
1     2957395
Name: count, dtype: int64

Statistical Summary for Each Cluster:
         event_type                                                    price  \
              count      mean       std  min  25%  50%  75%  max       count   
cluster                                                                        
0         7426282.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0   7426282.0   
1         2957395.0  1.239264  0.426634  1.0  1.0  1.0  1.0  2.0   2957395.0   
2        34483992.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  34483992.0   

                                                                            
                mean         std     min     25%     50%      75%      max  
cluster                                                                   